# Notebook 8: Out-of-Distribution (OOD) Detection

**Session 2: Failure Modes and Edge Cases**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/AV_Perception_Safety_Workshop/Session_2_Failure_Modes_and_Edge_Cases/notebooks/08_OOD_Detection.ipynb)

**Author:** Milin Patel  
**Duration:** ~25 minutes

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand the out-of-distribution (OOD) problem
- ✅ Implement Mahalanobis distance-based OOD detection
- ✅ Apply energy-based OOD detection
- ✅ Use Monte Carlo Dropout for uncertainty estimation
- ✅ Evaluate OOD detectors with AUROC and FPR@95
- ✅ Connect OOD detection to ISO 21448 SOTIF
- ✅ Design safety responses for OOD scenarios

---

## 📦 Setup and Imports

In [ ]:
# Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.covariance import EmpiricalCovariance
from scipy.spatial.distance import mahalanobis
import warnings
warnings.filterwarnings('ignore')

# Check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

print("✅ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")

---

## 1️⃣ The Out-of-Distribution Problem

### What is OOD?

**In-Distribution (ID):** Data similar to what the model was trained on  
**Out-of-Distribution (OOD):** Data that differs significantly from training data

### Examples in Autonomous Driving

**In-Distribution:**
- Cars, trucks, pedestrians, cyclists
- Clear weather, paved roads
- Standard traffic signs and markings

**Out-of-Distribution:**
- Unusual objects: Sofa on highway, escaped animals
- Extreme weather: Sandstorm, heavy snow, flooding
- Novel situations: Overturned truck, downed power lines
- Sensor degradation: Dirty camera, misaligned LiDAR

### Why OOD Detection Matters

**Problem:** Neural networks are **overconfident** on OOD inputs
- Model trained on cats/dogs may classify a horse as "dog" with 95% confidence
- AV model trained on clear weather may misclassify in heavy fog with high confidence

**Danger for AVs:**
- High confidence ≠ correctness
- System doesn't know what it doesn't know
- Can lead to catastrophic failures

**Solution:** OOD detection identifies when input is outside training distribution
- Flag uncertain situations
- Trigger safety response (slow down, hand over to driver, stop)

### Connection to ISO 21448 SOTIF

**SOTIF (Safety Of The Intended Functionality):**
- Addresses failures even when system works as designed
- Focus on **known and unknown unsafe scenarios**
- OOD detection helps identify **unknown unsafe scenarios**

**SOTIF Requirement:** System must detect when it's operating outside its ODD

In [ ]:
# Visualize ID vs OOD concept
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Generate synthetic data
np.random.seed(42)
# In-distribution: centered at (2, 2)
id_data = np.random.multivariate_normal([2, 2], [[1, 0.5], [0.5, 1]], 200)
# Out-of-distribution: far from ID data
ood_data = np.random.multivariate_normal([6, 6], [[0.5, 0], [0, 0.5]], 50)

# Plot 1: Feature space
axes[0].scatter(id_data[:, 0], id_data[:, 1], c='green', alpha=0.6, s=50, label='In-Distribution')
axes[0].scatter(ood_data[:, 0], ood_data[:, 1], c='red', alpha=0.6, s=50, label='Out-of-Distribution')
axes[0].set_xlabel('Feature 1', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Feature 2', fontsize=12, fontweight='bold')
axes[0].set_title('Feature Space: ID vs OOD', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Draw boundary
from matplotlib.patches import Ellipse
ellipse = Ellipse(xy=(2, 2), width=4, height=4, angle=45, 
                  facecolor='none', edgecolor='blue', linewidth=3, linestyle='--', label='ID Boundary')
axes[0].add_patch(ellipse)

# Plot 2: Confidence distribution
id_confidence = np.random.beta(8, 2, 200)  # High confidence for ID
ood_confidence = np.random.beta(5, 3, 50)  # Should be lower but often isn't!

axes[1].hist(id_confidence, bins=20, alpha=0.6, color='green', label='ID Confidence')
axes[1].hist(ood_confidence, bins=20, alpha=0.6, color='red', label='OOD Confidence (Problem!)')
axes[1].axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Threshold')
axes[1].set_xlabel('Model Confidence', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[1].set_title('Overconfidence Problem', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("⚠️ Problem: OOD data often receives high confidence scores!")
print("   Solution: OOD detection methods beyond softmax confidence")

---

## 2️⃣ Load Pre-trained Model and Data

We'll use CIFAR-10 as in-distribution and SVHN as out-of-distribution for demonstration.

In [ ]:
# Load pre-trained ResNet18 on CIFAR-10 (or train quickly)
print("Loading model...")

# Define simple ResNet for CIFAR-10
class SimpleCIFAR10Net(nn.Module):
    def __init__(self):
        super(SimpleCIFAR10Net, self).__init__()
        self.model = resnet18(pretrained=False, num_classes=10)
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()
    
    def forward(self, x):
        return self.model(x)
    
    def forward_features(self, x):
        """Return intermediate features for OOD detection"""
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)
        x = self.model.layer4(x)
        x = self.model.avgpool(x)
        x = torch.flatten(x, 1)
        return x

model = SimpleCIFAR10Net().to(device)

print("✅ Model loaded (for demo purposes, assume pre-trained)")
print("   In production, you'd train on your specific AV dataset")

In [ ]:
# Load datasets
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

print("Loading datasets...")

# In-distribution: CIFAR-10
id_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
id_loader = torch.utils.data.DataLoader(id_dataset, batch_size=100, shuffle=False, num_workers=2)

# Out-of-distribution: SVHN (street house numbers)
ood_dataset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform)
ood_loader = torch.utils.data.DataLoader(ood_dataset, batch_size=100, shuffle=False, num_workers=2)

print(f"✅ Datasets loaded:")
print(f"   ID (CIFAR-10): {len(id_dataset)} images")
print(f"   OOD (SVHN): {len(ood_dataset)} images")

# Show examples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i in range(5):
    axes[0, i].imshow(id_dataset[i][0].permute(1, 2, 0) * 0.5 + 0.5)
    axes[0, i].set_title(f'ID: {id_dataset.classes[id_dataset[i][1]]}', fontsize=10)
    axes[0, i].axis('off')
    
    axes[1, i].imshow(ood_dataset[i][0].permute(1, 2, 0) * 0.5 + 0.5)
    axes[1, i].set_title(f'OOD: SVHN', fontsize=10)
    axes[1, i].axis('off')

fig.suptitle('In-Distribution (CIFAR-10) vs Out-of-Distribution (SVHN)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 3️⃣ Method 1: Maximum Softmax Probability (Baseline)

**Idea:** Use maximum softmax probability as OOD score
- High probability → In-distribution
- Low probability → Out-of-distribution

**Problem:** As we'll see, this often fails!

In [ ]:
# Compute softmax scores
def compute_softmax_scores(model, loader, device):
    """Compute maximum softmax probability for each sample"""
    model.eval()
    scores = []
    
    with torch.no_grad():
        for images, _ in loader:
            images = images.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)
            max_probs, _ = torch.max(probs, dim=1)
            scores.extend(max_probs.cpu().numpy())
    
    return np.array(scores)

print("Computing Maximum Softmax Probability scores...")
id_softmax_scores = compute_softmax_scores(model, id_loader, device)
ood_softmax_scores = compute_softmax_scores(model, ood_loader, device)

print(f"✅ Scores computed:")
print(f"   ID mean: {id_softmax_scores.mean():.3f} ± {id_softmax_scores.std():.3f}")
print(f"   OOD mean: {ood_softmax_scores.mean():.3f} ± {ood_softmax_scores.std():.3f}")

# Visualize distributions
plt.figure(figsize=(10, 5))
plt.hist(id_softmax_scores, bins=50, alpha=0.6, color='green', label='ID', density=True)
plt.hist(ood_softmax_scores, bins=50, alpha=0.6, color='red', label='OOD', density=True)
plt.xlabel('Maximum Softmax Probability', fontsize=12, fontweight='bold')
plt.ylabel('Density', fontsize=12, fontweight='bold')
plt.title('Baseline: Maximum Softmax Probability', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n⚠️ Notice: Significant overlap! OOD samples still get high confidence.")

---

## 4️⃣ Method 2: Mahalanobis Distance-Based Detection

**Idea:** Measure distance in feature space using Mahalanobis distance

**Mahalanobis Distance:**
$$D_M(x) = \sqrt{(x - \mu)^T \Sigma^{-1} (x - \mu)}$$

Where:
- $\mu$ = mean of ID features
- $\Sigma$ = covariance matrix of ID features

**Intuition:** How far is this sample from the typical ID distribution?

**Reference:** Lee et al. (2018) - "A Simple Unified Framework for Detecting OOD Examples"

In [ ]:
# Extract features from ID training data
def extract_features(model, loader, device, max_samples=1000):
    """Extract penultimate layer features"""
    model.eval()
    features = []
    
    with torch.no_grad():
        for i, (images, _) in enumerate(loader):
            if i * loader.batch_size >= max_samples:
                break
            images = images.to(device)
            feats = model.forward_features(images)
            features.append(feats.cpu().numpy())
    
    return np.concatenate(features, axis=0)

print("Extracting features from ID data...")
id_train_features = extract_features(model, id_loader, device, max_samples=2000)

# Fit Gaussian distribution
mean = np.mean(id_train_features, axis=0)
cov = EmpiricalCovariance().fit(id_train_features)

print(f"✅ Fitted Gaussian:")
print(f"   Feature dimension: {mean.shape[0]}")
print(f"   Number of samples: {id_train_features.shape[0]}")

In [ ]:
# Compute Mahalanobis distance scores
def compute_mahalanobis_scores(model, loader, device, mean, cov):
    """Compute Mahalanobis distance for each sample"""
    model.eval()
    scores = []
    
    with torch.no_grad():
        for images, _ in loader:
            images = images.to(device)
            features = model.forward_features(images).cpu().numpy()
            
            for feat in features:
                score = mahalanobis(feat, mean, cov.precision_)
                scores.append(score)
    
    return np.array(scores)

print("Computing Mahalanobis distance scores...")
id_mahal_scores = compute_mahalanobis_scores(model, id_loader, device, mean, cov)
ood_mahal_scores = compute_mahalanobis_scores(model, ood_loader, device, mean, cov)

print(f"✅ Scores computed:")
print(f"   ID mean: {id_mahal_scores.mean():.3f} ± {id_mahal_scores.std():.3f}")
print(f"   OOD mean: {ood_mahal_scores.mean():.3f} ± {ood_mahal_scores.std():.3f}")

# Visualize distributions
plt.figure(figsize=(10, 5))
plt.hist(id_mahal_scores, bins=50, alpha=0.6, color='green', label='ID', density=True)
plt.hist(ood_mahal_scores, bins=50, alpha=0.6, color='red', label='OOD', density=True)
plt.xlabel('Mahalanobis Distance', fontsize=12, fontweight='bold')
plt.ylabel('Density', fontsize=12, fontweight='bold')
plt.title('Mahalanobis Distance-Based OOD Detection', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n✅ Better separation! OOD samples have higher distance.")

---

## 5️⃣ Method 3: Energy-Based Detection

**Idea:** Use energy function instead of softmax

**Energy:**
$$E(x) = -\log \sum_{i=1}^{C} e^{f_i(x)}$$

Where $f_i(x)$ is the logit for class $i$.

**Intuition:** 
- ID samples: Low energy (model is confident)
- OOD samples: High energy (model is uncertain)

**Reference:** Liu et al. (2020) - "Energy-based Out-of-distribution Detection"

In [ ]:
# Compute energy scores
def compute_energy_scores(model, loader, device, temperature=1.0):
    """Compute energy score for each sample"""
    model.eval()
    scores = []
    
    with torch.no_grad():
        for images, _ in loader:
            images = images.to(device)
            logits = model(images)
            # Energy = -log(sum(exp(logits)))
            energy = -temperature * torch.logsumexp(logits / temperature, dim=1)
            scores.extend(energy.cpu().numpy())
    
    return np.array(scores)

print("Computing Energy scores...")
id_energy_scores = compute_energy_scores(model, id_loader, device)
ood_energy_scores = compute_energy_scores(model, ood_loader, device)

print(f"✅ Scores computed:")
print(f"   ID mean: {id_energy_scores.mean():.3f} ± {id_energy_scores.std():.3f}")
print(f"   OOD mean: {ood_energy_scores.mean():.3f} ± {ood_energy_scores.std():.3f}")

# Visualize distributions
plt.figure(figsize=(10, 5))
plt.hist(id_energy_scores, bins=50, alpha=0.6, color='green', label='ID', density=True)
plt.hist(ood_energy_scores, bins=50, alpha=0.6, color='red', label='OOD', density=True)
plt.xlabel('Energy Score', fontsize=12, fontweight='bold')
plt.ylabel('Density', fontsize=12, fontweight='bold')
plt.title('Energy-Based OOD Detection', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n✅ Good separation! OOD samples have higher (less negative) energy.")

---

## 6️⃣ Method 4: Monte Carlo Dropout Uncertainty

**Idea:** Use dropout at inference time to estimate uncertainty

**Procedure:**
1. Keep dropout enabled during inference
2. Run multiple forward passes (e.g., 30 times)
3. Compute variance of predictions
4. High variance → High uncertainty → Likely OOD

**Advantages:**
- No additional training required
- Provides uncertainty estimates
- Applicable to existing models

**Reference:** Gal & Ghahramani (2016) - "Dropout as a Bayesian Approximation"

In [ ]:
# Add dropout to model (for demonstration)
class ModelWithDropout(nn.Module):
    def __init__(self, base_model, dropout_rate=0.3):
        super(ModelWithDropout, self).__init__()
        self.base_model = base_model
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        features = self.base_model.forward_features(x)
        features = self.dropout(features)  # Apply dropout
        output = self.base_model.model.fc(features)
        return output

dropout_model = ModelWithDropout(model).to(device)

def compute_mc_dropout_uncertainty(model, loader, device, num_samples=10):
    """Compute predictive uncertainty using MC Dropout"""
    model.train()  # Keep dropout enabled
    uncertainties = []
    
    with torch.no_grad():
        for images, _ in loader:
            images = images.to(device)
            
            # Multiple forward passes
            predictions = []
            for _ in range(num_samples):
                outputs = model(images)
                probs = F.softmax(outputs, dim=1)
                predictions.append(probs.cpu().numpy())
            
            # Stack predictions
            predictions = np.stack(predictions, axis=0)  # Shape: [num_samples, batch, classes]
            
            # Compute variance (uncertainty)
            variance = np.var(predictions, axis=0).max(axis=1)  # Max variance across classes
            uncertainties.extend(variance)
    
    return np.array(uncertainties)

print("Computing MC Dropout uncertainty (this may take a moment)...")
id_mc_uncertainty = compute_mc_dropout_uncertainty(dropout_model, id_loader, device, num_samples=10)
ood_mc_uncertainty = compute_mc_dropout_uncertainty(dropout_model, ood_loader, device, num_samples=10)

print(f"✅ Uncertainty computed:")
print(f"   ID mean: {id_mc_uncertainty.mean():.4f} ± {id_mc_uncertainty.std():.4f}")
print(f"   OOD mean: {ood_mc_uncertainty.mean():.4f} ± {ood_mc_uncertainty.std():.4f}")

# Visualize distributions
plt.figure(figsize=(10, 5))
plt.hist(id_mc_uncertainty, bins=50, alpha=0.6, color='green', label='ID', density=True)
plt.hist(ood_mc_uncertainty, bins=50, alpha=0.6, color='red', label='OOD', density=True)
plt.xlabel('MC Dropout Uncertainty (Variance)', fontsize=12, fontweight='bold')
plt.ylabel('Density', fontsize=12, fontweight='bold')
plt.title('Monte Carlo Dropout Uncertainty', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n✅ OOD samples show higher uncertainty!")

---

## 7️⃣ Evaluation: AUROC and FPR@95

### Metrics for OOD Detection

**AUROC (Area Under ROC Curve):**
- Measures overall detection performance
- 1.0 = perfect separation, 0.5 = random
- **Target for safety-critical systems: > 0.95**

**FPR@95 (False Positive Rate at 95% TPR):**
- What % of ID samples are wrongly flagged as OOD?
- Lower is better (fewer false alarms)
- **Target: < 5%**

**Why these metrics?**
- AVs must detect OOD reliably (high TPR)
- But minimize false alarms (low FPR) to avoid unnecessary interventions

In [ ]:
# Evaluation function
def evaluate_ood_detector(id_scores, ood_scores, higher_is_ood=True):
    """
    Evaluate OOD detector using AUROC and FPR@95
    
    Args:
        id_scores: Scores for in-distribution samples
        ood_scores: Scores for out-of-distribution samples
        higher_is_ood: If True, higher scores indicate OOD
    """
    # Create labels
    y_true = np.concatenate([np.zeros(len(id_scores)), np.ones(len(ood_scores))])
    
    # Create predictions
    if higher_is_ood:
        y_score = np.concatenate([id_scores, ood_scores])
    else:
        y_score = np.concatenate([-id_scores, -ood_scores])
    
    # Compute AUROC
    auroc = roc_auc_score(y_true, y_score)
    
    # Compute FPR@95
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    fpr_at_95 = fpr[np.argmax(tpr >= 0.95)]
    
    return auroc, fpr_at_95, fpr, tpr

# Evaluate all methods
methods = {
    'Maximum Softmax': (id_softmax_scores, ood_softmax_scores, False),  # Lower is OOD
    'Mahalanobis Distance': (id_mahal_scores, ood_mahal_scores, True),
    'Energy-Based': (id_energy_scores, ood_energy_scores, True),
    'MC Dropout': (id_mc_uncertainty, ood_mc_uncertainty, True)
}

results = []
roc_curves = {}

for method_name, (id_sc, ood_sc, higher_ood) in methods.items():
    auroc, fpr95, fpr, tpr = evaluate_ood_detector(id_sc, ood_sc, higher_ood)
    results.append({
        'Method': method_name,
        'AUROC': auroc,
        'FPR@95': fpr95
    })
    roc_curves[method_name] = (fpr, tpr)

results_df = pd.DataFrame(results)
display(results_df.style.background_gradient(cmap='RdYlGn', subset=['AUROC']).format(precision=3))

print("\n📊 Performance Comparison:")
print(f"   Best AUROC: {results_df.loc[results_df['AUROC'].idxmax(), 'Method']} ({results_df['AUROC'].max():.3f})")
print(f"   Best FPR@95: {results_df.loc[results_df['FPR@95'].idxmin(), 'Method']} ({results_df['FPR@95'].min():.3f})")

In [ ]:
# Plot ROC curves
plt.figure(figsize=(10, 8))
colors = ['blue', 'green', 'red', 'purple']

for (method_name, (fpr, tpr)), color in zip(roc_curves.items(), colors):
    auroc = results_df[results_df['Method'] == method_name]['AUROC'].values[0]
    plt.plot(fpr, tpr, linewidth=2, color=color, label=f'{method_name} (AUROC={auroc:.3f})')

# Diagonal line (random classifier)
plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random')

# Mark TPR=0.95 point
plt.axhline(y=0.95, color='gray', linestyle='--', alpha=0.5)
plt.text(0.7, 0.96, 'TPR = 0.95', fontsize=10)

plt.xlabel('False Positive Rate', fontsize=12, fontweight='bold')
plt.ylabel('True Positive Rate', fontsize=12, fontweight='bold')
plt.title('ROC Curves: OOD Detection Methods', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("   - Curve closer to top-left = better performance")
print("   - Mahalanobis and Energy-based outperform baseline Softmax")
print("   - MC Dropout provides uncertainty with minimal changes")

---

## 8️⃣ Safety Implications and ISO 21448 SOTIF

### Connection to SOTIF

**ISO 21448 SOTIF** addresses:
1. **Known unsafe scenarios:** Identified during testing
2. **Unknown unsafe scenarios:** Not yet identified

**OOD Detection helps with:**
- **Detecting unknown scenarios** at runtime
- **Triggering safety responses** (MRC - Minimum Risk Condition)
- **Expanding scenario coverage** by logging OOD cases

### Safety Response Protocol

When OOD is detected:

1. **Alert driver** (Level 2/3) or **initiate fallback** (Level 4/5)
2. **Reduce speed** gradually
3. **Increase following distance**
4. **Activate emergency lights** if pulling over
5. **Log incident** for offline analysis
6. **Do NOT** make aggressive maneuvers

### Performance Requirements

For safety-critical AV deployment:
- **AUROC > 0.95:** Reliable detection
- **FPR@95 < 0.05:** Minimize false alarms (< 5%)
- **Latency < 100ms:** Real-time detection
- **Robustness:** Works across weather, lighting, sensor degradation

In [ ]:
# Safety decision logic example
def safety_decision(ood_score, threshold, speed_kmh):
    """
    Make safety decision based on OOD score
    
    Args:
        ood_score: OOD detection score (higher = more OOD)
        threshold: OOD threshold
        speed_kmh: Current speed in km/h
    
    Returns:
        action: Safety action to take
    """
    if ood_score < threshold * 0.7:
        return "NORMAL: Continue autonomous operation"
    elif ood_score < threshold:
        return "CAUTION: Reduce speed by 20%, increase following distance"
    elif ood_score < threshold * 1.5:
        return "WARNING: Reduce speed by 50%, alert driver/remote operator"
    else:
        return "CRITICAL: Initiate Minimum Risk Condition (pull over safely)"

# Example scenarios
threshold = 50.0  # Example threshold
scenarios = [
    (20, "Clear day, normal traffic"),
    (40, "Light fog, slightly reduced visibility"),
    (55, "Heavy fog, unusual object detected"),
    (85, "Extreme weather, multiple unknown objects")
]

print("Safety Decision Examples:\n" + "="*70)
for score, description in scenarios:
    action = safety_decision(score, threshold, speed_kmh=50)
    print(f"\nScenario: {description}")
    print(f"OOD Score: {score:.1f}")
    print(f"Action: {action}")

print("\n" + "="*70)
print("💡 Graded response: Severity of action matches uncertainty level")

---

## ✏️ Exercise: Design OOD Detector

**Task:** You're designing an OOD detector for a Level 4 campus shuttle.

**In-Distribution Scenarios:**
- Students walking, bicycles, campus buses
- Paved roads, crosswalks, stop signs
- Clear weather, daylight hours
- Speed: 15-25 km/h

**Questions:**
1. What OOD scenarios might occur?
2. Which OOD detection method would you choose and why?
3. What AUROC and FPR@95 targets would you set?
4. What safety response would you implement for OOD detection?
5. How would you validate your OOD detector?

In [ ]:
# TODO: Complete your design

your_design = {
    'ood_scenarios': [
        # List potential OOD scenarios
        # Example: "Construction zone with barriers"
    ],
    'chosen_method': '',  # e.g., "Mahalanobis Distance + MC Dropout ensemble"
    'rationale': '',  # Why did you choose this method?
    'auroc_target': 0.0,  # Your AUROC target
    'fpr95_target': 0.0,  # Your FPR@95 target
    'safety_response': '',  # What action when OOD detected?
    'validation_plan': [
        # How will you test the OOD detector?
    ]
}

print("Your OOD Detector Design:")
print(f"\nChosen Method: {your_design['chosen_method']}")
print(f"Rationale: {your_design['rationale']}")
print(f"\nPerformance Targets:")
print(f"  AUROC: {your_design['auroc_target']}")
print(f"  FPR@95: {your_design['fpr95_target']}")
print(f"\nSafety Response: {your_design['safety_response']}")

print("\n🤔 Reflection:")
print("   - How does this relate to ISO 21448 SOTIF?")
print("   - What are the trade-offs of your chosen method?")
print("   - How would you handle false positives?")

---

## 🎯 Key Takeaways

### The OOD Problem
- **Neural networks are overconfident** on out-of-distribution inputs
- **Danger:** High confidence doesn't mean correct prediction
- **Solution:** OOD detection methods beyond softmax probability

### OOD Detection Methods
1. **Maximum Softmax (Baseline):** Simple but often fails
2. **Mahalanobis Distance:** Measures distance in feature space (effective!)
3. **Energy-Based:** Uses energy function (strong performance)
4. **MC Dropout:** Estimates uncertainty (minimal model changes)

### Evaluation Metrics
- **AUROC:** Overall detection performance (target > 0.95)
- **FPR@95:** False alarm rate (target < 0.05)

### Safety Implications
- **ISO 21448 SOTIF:** OOD detection addresses unknown unsafe scenarios
- **Safety response:** Graded actions based on OOD score
- **Requirements:** High AUROC, low FPR, real-time performance

### Best Practices
1. ✅ Use **multiple OOD methods** (ensemble)
2. ✅ Set **conservative thresholds** (safety over efficiency)
3. ✅ Implement **graded responses** (not binary)
4. ✅ **Log all OOD detections** for offline analysis
5. ✅ **Continuously update** training data with OOD discoveries

---

## 🔜 Next: Corner Cases and Edge Cases

Now let's explore specific types of challenging scenarios in autonomous driving!

**Ready?** Open `09_Corner_Cases_and_Edge_Cases.ipynb`

---

*Notebook created by Milin Patel | Hochschule Kempten*  
*Last updated: 2025-01-18*